In [1]:
import pandas as pd
import datetime as dt
import statsmodels.api as sm
from statsmodels.api import add_constant
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import ipywidgets as widgets
from ipywidgets import *
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
import seaborn as sns
from scipy import stats
from scipy.stats import t
from scipy.stats import chi2
from scipy.stats import norm
from numpy import average, std
from math import sqrt
import matplotlib.pyplot as plot
import scipy.stats
%matplotlib inline
from IPython.display import display, Markdown

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

In [2]:
url = 'https://github.com/AbdulrahmanAlFawzan1/Widnow_Dressing/raw/master/average_q_prices.csv'
aq = pd.read_csv(url)
aq['date'] = pd.to_datetime(aq['date'])
aq['date']=aq['date'].map(dt.datetime.toordinal)

In [3]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
def printmd(string):
    display(Markdown(string))

In [4]:
def lin_reg(a):
    
    X = aq["date"] # independent variable
    y = aq[a] # dependent variable
    X = sm.add_constant(X) # y intrecept
    model = sm.OLS(y, X).fit() # fitting the model
    predictions = model.predict(X)
        
    binshistogram=int(round(sqrt(len(y)),0))
    
    f, axes = plt.subplots(2, 2,figsize=(20,10))
    sns.residplot(x='date', y=a, data=aq, ax=axes[0,0]).set_title("Residuals plot")
    sns.regplot(x='date', y=a, data=aq,ax=axes[0,1]).set_title("Data Scatter Plot and Regression Line")
    sns.distplot(y,bins=binshistogram, ax = axes[1,0])
    axes[1,0].set_title("Histogram")
    axes[1,0].set_xlabel('Return')
    axes[1,0].set_ylabel('Frequency')
    scipy.stats.probplot(aq[a],plot=axes[1,1])
    axes[1,1].set_title("Normal Probablity Plot")
    axes[1,1].set_xlim(-2.5,2.5)
    axes[1,1].set_ylim(-0.007,0.008)
    return(model.summary())

def meanconf(a,b):
    X = aq[a]
    c = (100-b)/100
    conf=c
    conff=100*conf
    printmd('* Mean:')
    mean = average(X)
    stddev = std(X, ddof=1)
    t_bounds = t.interval(conf, len(X) - 1)
    ci = [mean + critval * stddev / sqrt(len(X)) for critval in t_bounds]
    print("Mean of sample: " + color.BOLD + "%f" % mean + color.END)
    print("Confidence Interval of the mean " + color.RED + "%.2f%%" % conff + color.END + ":" + color.BOLD + "(%f, %f)" % (ci[0], ci[1])+ color.END)
    return""

def varianceconf(a,b):
    X = aq[a]
    c = (100-b)/100
    conf=c
    conff=100*conf
    printmd('* Variance:')
    stddev = std(X, ddof=1)
    variance = stddev * stddev
    chi2_bounds = chi2.interval(conf, len(X),loc=0, scale=1)
    ci = [len(X) * variance/critval for critval in chi2_bounds]
    print("Variance of sample: " + color.BOLD + "%f" % variance + color.END)
    print("Confidence Interval of the variance " + color.RED + "%.2f%%" % conff + color.END + ":" + color.BOLD + ": (%f, %f)" % (ci[0], ci[1])+ color.END)
    return""


In [5]:
output_stock = widgets.Output()

# defining the dropdown menu
dropdown_stock = widgets.Dropdown(options =aq.columns[1:],value=None, description='Stock:', disabled=False)
slider = widgets.IntSlider(value=None, min=1, max=99, step=1,description='Confidence Level %',
                                    disabled=False,continuous_update=False,orientation='horizontal',
                                    style = {'description_width': 'initial'},layout={'width': '400px'})


def dropdown_stock_eventhandler(change):
        
    output_stock.clear_output()
    
    with output_stock:
        printmd('## Analysis of one stock:')
        printmd('### Linear regression:')
        display(lin_reg(dropdown_stock.value))
        printmd('### Confidence intervals:')
        display(meanconf(dropdown_stock.value,change.new))
        display(varianceconf(dropdown_stock.value,change.new))
        printmd('### Plots:')
        show_inline_matplotlib_plots()
        

#         

#pairing the dropdown menu with updates with the function
slider.observe(dropdown_stock_eventhandler, names='value')
#displaying the results
#display(dropdown_stock)
#o1 =display(output_stock)

In [6]:
def lin_reg2(a,b):
    
    X = aq[a] # independent variable
    y = aq[b] # dependent variable
    X = sm.add_constant(X) # y intrecept
    model = sm.OLS(y, X).fit() # fitting the model
    predictions = model.predict(X)
    
    f, axes = plt.subplots(1, 2,figsize=(20,5))
    sns.residplot(x=a, y=b, data=aq, ax=axes[0]).set_title("Residuals plot")
    sns.regplot(x=a, y=b, data=aq,ax=axes[1]).set_title("Data Scatter Plot and Regression Line")
    return(model.summary())

def eqmeans(a,b,c):
    X = aq[a]
    Y = aq[b]
    conf=(100-c)/100
    meanX=average(X)
    meanY=average(Y)
    stddevX = std(X, ddof=1)
    stddevY = std(Y, ddof=1)
    norm_bounds = norm.interval(conf, loc=0, scale=1)
    param = ((meanX-meanY)/sqrt((pow(stddevX,2)/len(X))+(pow(stddevY,2)/len(Y))))
    
    print("Equality of means of two normal samples:\n")
    
    print("H\N{SUBSCRIPT ZERO}: "+ u'\u00b5'+ " " + a + " = "+ u'\u00b5' + " " +b + ".")
    print("H\N{SUBSCRIPT ONE}: "+ u'\u00b5'+ " " + a + " " + u'\u2260' + " " + u'\u00b5' + " " +b + ".\n")
    if norm_bounds[0]<param<norm_bounds[1]:
        print(color.BOLD + "Test Result : \n\nDo not reject H\N{SUBSCRIPT ZERO}." + color.END)
    else:
        print(color.BOLD + "Reject H\N{SUBSCRIPT ZERO}." + color.END)
    
    return "End of analysis"
        

# defining the dropdown menu
dropdown_stock1 = widgets.Dropdown(options =aq.columns[1:],value=None, description='Choose first stock:', 
                                   disabled=False,continuous_update=False
                                   ,style={'description_width': 'initial'},layout={'width': 'initial'})

dropdown_stock2 = widgets.Dropdown(options =aq.columns[1:],value=None, description='Choose second stock:',
                                   disabled=False,continuous_update=False,
                                   style={'description_width': 'initial'},layout={'width': 'initial'})

slider1 = widgets.IntSlider(value=None, min=1, max=99, step=1,description='Confidence Level %',
                                    disabled=False,continuous_update=False,orientation='horizontal',
                                    style = {'description_width': 'initial'},layout={'width': '400px'})


output = widgets.Output()


def options(a,b,c):
    output.clear_output()
    
    with output:
        printmd('## Comparison of two stocks:')
        printmd('### Regression of one stock on the other:')
        display(lin_reg2(a,b))
        printmd('### Test of equality of means:')
        display(eqmeans(a,b,c))
        printmd('### Plots:')
        show_inline_matplotlib_plots()
        

    
# def dropdown_stock_eventhandler1(change):
#     options(change.new,dropdown_stock2.value)

# def dropdown_stock_eventhandler2(change):
#     options(dropdown_stock1.value,change.new)
    
def slider_eventhandler3(change):
    options(dropdown_stock1.value,dropdown_stock2.value,change.new)

#pairing the dropdown menu with updates with the function
#dropdown_stock1.observe(dropdown_stock_eventhandler1, names='value')
#dropdown_stock2.observe(dropdown_stock_eventhandler2, names='value')
slider1.observe(slider_eventhandler3, names='value')
#displaying the results
#display(dropdown_stock1)
#display(dropdown_stock2)
#o2 = display(output)   

In [7]:
    

%%html
<style>
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    flex: 0 1 250px
}
</style>

In [8]:
tab1 = VBox(children=[dropdown_stock,slider,output_stock])

pre_tab2=HBox(children=[dropdown_stock1,dropdown_stock2])
tab2 = VBox(children=[pre_tab2,slider1,output,])
tab = widgets.Tab([tab1,tab2])
tab.set_title(0, 'Single Stock Analysis')
tab.set_title(1, 'Two Stocks Comparision') 

display(tab)
output_stock.clear_output()
output.clear_output()